<a href="https://colab.research.google.com/github/1e16miin/project1/blob/main/session1_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

지역에 따라서 선호하는 게임 장르가 다를까 라는 질문에 대답을 하셔야합니다.//group by해서 chi2

연도별 게임의 트렌드가 있을까 라는 질문에 대답을 하셔야합니다.// 범주형 변수? chi2

출고량이 높은 게임에 대한 분석 및 시각화 프로세스가 포함되어야 합니다.

In [1]:
import pandas as pd
import numpy as np


df = pd.read_csv("vgames2.csv", na_values="N/A")
df = df.drop(["Unnamed: 0"], axis = 1)

In [2]:
# df = df.set_index("Name", drop=True)

In [2]:
for col in df.columns:
  print(col, df[col].isnull().sum())

Name 0
Platform 0
Year 271
Genre 50
Publisher 58
NA_Sales 0
EU_Sales 0
JP_Sales 0
Other_Sales 0


년도와 Genre 에서만 na_value가 존재한다.

In [3]:
from numpy.core.numeric import NaN

def preprocess_year(year):
  try:
    if year < 100:  
      if year <= 21:
        year += 2000
      else:
        year += 1900
    
    return int(year)
  except:
    return 0

def fill_genre(genre):
  if genre is NaN:
    return "MISC"
  else:
    return genre

def fill_publisher(publisher):
  if publisher is NaN:
    return "Unknown"
  else:
    return publisher


df.Year = df.Year.apply(preprocess_year)
df.Genre = df.Genre.apply(fill_genre)
df.Publisher = df.Publisher.apply(fill_publisher)

In [82]:
from bs4 import BeautifulSoup
from urllib.parse import quote_plus
import requests
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
import time

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]


In [83]:
def fill_year(name):
  try:
    idx = name.find("(")
    if idx != -1:
      name = name[:idx]
    baseUrl = 'https://www.google.com/search?q='
    plusUrl = quote_plus(name)
    url = baseUrl + plusUrl
    driver.get(url)

    html = driver.page_source
    soup = BeautifulSoup(html)
    date = soup.select_one('.Eq0J8.LrzXr.kno-fv').text
    year = date.split(" ")[-1]
    return int(year)
  except:
    return NaN

In [74]:
fill_year("yu-gi-oh! 5d's wheelie breakers")

-1


2009

In [16]:
def preprocess_sales(numeric):#단위를 모두 K로 변환
  try:
    if numeric.isdigit():
      numeric *= 1e3
    else:
      if "K" in numeric:
        numeric = float(numeric.replace("K", ""))
      else:
        numeric = float(numeric.replace("M", ""))*1e3
    return numeric
  except:
    return 0
for sales in df.columns[-4:]:
  df[sales] = df[sales].apply(preprocess_sales)

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

In [ ]:
# year_platform = df[["Year", "Platform"]].dropna()

In [ ]:
# year_platform_crosstab = pd.crosstab(df.Platform, df.Year, dropna=True, margins=False)


In [ ]:
# year_platform_crosstab

Year,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2020.0
Platform,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2600,9,46,36,11,1,1,2,6,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3DO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3DS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,116,93,91,79,86,35,0,0
DC,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,14,20,8,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0
DS,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,23,118,202,377,492,405,326,153,25,9,1,0,0,0,1
GB,0,0,0,0,0,0,0,0,1,10,3,9,7,0,3,4,6,8,8,11,17,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
GBA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,107,198,150,175,135,39,6,0,0,0,0,0,0,0,0,0,0,0
GC,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22,152,144,83,97,40,4,0,0,0,0,0,0,0,0,0,0,0
GEN,0,0,0,0,0,0,0,0,0,0,1,1,6,7,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# def fill_year(platform):
#   return year_platform_crosstab.loc[platform].idxmax()

# df.Year = df.apply(lambda row: fill_year(row["Platform"], row["Year"]), axis=1)
# df.Year = df.Year.fillna(df.index.to_series().apply(fill_year))

In [84]:
temp = df[["Name","Year"]].apply(lambda row: fill_year(row[0]) if row[1] == 0 else row[1], axis=1)

In [85]:
temp.loc[temp.isnull()]

1235    NaN
1301    NaN
1572    NaN
1796    NaN
2056    NaN
         ..
16240   NaN
16275   NaN
16278   NaN
16425   NaN
16432   NaN
Length: 229, dtype: float64

In [48]:
df.loc[df.Year == 0]

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
31,Rock Band,X360,0,Misc,Electronic Arts,1.93,0.34,0,0.21
109,Yu-Gi-Oh! 5D's Wheelie Breakers (JP sales),Wii,0,Racing,Unknown,0,0,0.02,0
273,Jewel Link Chronicles: Mountains of Madness,DS,0,Puzzle,Avanquest,0,0.06,0,0.01
358,Majesty 2: The Fantasy Kingdom Sim,X360,0,Simulation,Unknown,0.03,0,0,0
430,Famista 64,N64,0,Sports,Namco Bandai Games,0,0,0.17,0.03
...,...,...,...,...,...,...,...,...,...
16240,Freaky Flyers,GC,0,Racing,Unknown,0.01,0,0,0
16275,Space Invaders,2600,0,Shooter,Atari,2.36,0.14,0M,0.03
16278,Madden NFL 11,Wii,0,Sports,Unknown,0.7,0,0,50K
16425,Yoostar on MTV,X360,0,Misc,Unknown,0.06,0,0,0
